#Data 06: 따릉이

서울 열린데이터 광장에서 제공하는 서울시 공공자전거 이용현황 데이터.  

* **Data1 : 서울특별시 공공자전거 대여소 정보**  
서울특별시 공공자전거 대여소 정보  
서울특별시 공공자전거 대여소(따릉이) 현황정보입니다.  
대여소의 이름, 관리번호, 위치정보와, 거치대수 정보를 제공합니다.


* **Data2 : 서울특별시 공공자전거 대여소별 이용정보(월별)**  
서울특별시 공공자전거 대여소별 대여, 반납 정보입니다.  
년월, 대여소번호, 대여소명, 대여건수, 반납건수 정보를 제공합니다.  
(Data2의 경우, 대여소그룹 컬럼을 제거한 것을 활용)

* **Data3 : 서울특별시 공공자전거 이용정보(시간대별)**
서울특별시 공공자전거 시간대별 이용정보입니다.  
대여일시, 대여시간, 대여소번호, 대여소명, 정기권유무, 성별, 연령대, 탄소량, 이동거리, 이동시간 정보를 제공합니다.

이용건수: 해당 시간대에 대여한 건수의 합  
그룹명 없음, 대여소명 없음, 정비센터 등인 경우 테스트 및 정비 목적이므로 분석에 제외하여 활용할 것

* Data from:  서울 열린데이터 광장 (http://data.seoul.go.kr/dataList/OA-14994/F/1/datasetView.do)



---



##1.데이터 둘러보기

In [ ]:
#한글 폰트 설정하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
#런타임 다시 시작

In [ ]:
#기본 패키지 불러오기
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('seaborn')
sns.set(font_scale=1)
plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['font.family'] = 'NanumGothic'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#구글 드라이브 마운트하기(이미 되어있다면 건너뛴다)
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
#데이터 불러오기
#한글 데이터가 있기 때문에 인코딩 문제 발생 > cp949 or euc-kr 사용
bike1 = pd.read_csv('/content/drive/MyDrive/Python/06_Ddareungee/공공자전거 대여소 정보(21.01.31 기준).csv', encoding='cp949')
bike2 = pd.read_csv('/content/drive/MyDrive/Python/06_Ddareungee/공공자전거 대여소별 이용정보_2020.07_2021.01.csv', encoding='cp949')
bike3 = pd.read_csv('/content/drive/MyDrive/Python/06_Ddareungee/공공자전거 이용정보(시간대별)_21.01.csv', encoding='cp949')

In [ ]:
#데이터 확인하기
bike3.head()

In [ ]:
#데이터 기본 정보 확인하기
bike1.info()

In [ ]:
#기본 통계량 확인하기
bike1.describe()

In [ ]:
#데이터 타입이 object인 것들도 확인
bike3.describe(include=np.object)

##2.데이터를 보고 질문 만들기

* 이동거리/평균사용시간이 높은 대여소는?
* 운동량과 이동거리의 상관관계는?
* 따릉이를 가장 많이 이용하는 성별은?
* 서울시 구별 설치 거치대의 개수는?
* 서울시 구별로 거치대 개수와 대여건수는?



##3.데이터 정비하기

###3-1.컬럼명 바꾸기

In [ ]:
bike1.head()

In [ ]:
#컬럼명 변경하기
bike1.columns = ['대여소번호', '대여소명', '자치구', '상세주소', '위도', '경도', '설치시기', '거치대수lcd', '거치대수qr', '운영방식']

In [ ]:
bike2.head()

###3-2. 필요없는 컬럼 제거하기

In [ ]:
bike1.head()

In [ ]:
#상세주소, 위도, 경도 제거하기
bike1 = bike1.drop(['상세주소','위도', '경도'], axis=1, inplace=False)
#2, 3은 제거할 만한 컬럼 없음!

###3-3. NaN 값 처리하기

In [ ]:
#LCD, QR 거치대 수에 NaN 값은 0으로 처리하자
bike1.head()

In [ ]:
bike1['거치대수lcd'] = bike1['거치대수lcd'].fillna(0)
bike1['거치대수qr'] = bike1['거치대수qr'].fillna(0)

###3-4. 대여소명 바꾸기

In [ ]:
#대여소명에서 숫자 떼버리기
#어떻게?
#대여소명은 숫자 + . + 대여소 이름 
bike3['대여소명'].str.split(".", expand = True).head()

In [ ]:
#새 컬럼을 만들자!
bike3['대여소명_new'] = bike3['대여소명'].str.split(".", expand = True)[1]

In [ ]:
#새 컬럼 확인
bike3.head()

In [ ]:
#bike2에도 같은 작업
bike2['대여소번호'] = bike2['대여소 명'].str.split(".", expand = True)[0]
bike2['대여소명_new'] = bike2['대여소 명'].str.split(".", expand = True)[1]

In [ ]:
#정비센터에는 대여소 번호가 없기 때문에 대여소번호에 한글이 들어가는 경우가 발생한다
bike2[bike2['대여소명_new'].isnull()]

In [ ]:
#분석에서 제외하기 위해 drop 시켜주자
bike22 = bike2.dropna(subset=['대여소명_new'])

In [ ]:
bike22

###3-5. 성별 정리하기

In [ ]:
#성별 컬럼에 대소문자, NaN이 섞여있다
print('성별: ', bike3['성별'].unique())

In [ ]:
#어느 index에 이상한 값이 들어있는가?
#행 index 가져오기
print(bike3.index[bike3['성별'] == 'f'].tolist())

In [ ]:
#성별 카테고리 정리하기
bike3['성별'][(bike3['성별'] == 'f')] = 'F'
bike3['성별'][(bike3['성별'] == 'm')] = 'M'
print('성별: ', bike3['성별'].unique())
#NaN 값은 어떻게 처리할까?
#의미있는 다른 값으로 변경하기가 어렵고, 다른 값으로 변경해서 데이터 처리가 편해질 것 같지 않다.

###3-6. 컬럼의 데이터타입 변환하기

In [ ]:
#대여소번호 데이터타입을 objdct로 바꾸기
print(bike1['대여소번호'].dtypes)

In [ ]:
bike1['대여소번호'] = bike1['대여소번호'].astype(object)

In [ ]:
#운동량, 탄소량 데이터타입을 numeric으로 바꾸기
print(bike3['운동량'].dtypes)
print(bike3['탄소량'].dtypes)

In [ ]:
#바로 실행하면 오류가 난다
bike3['운동량'] = pd.to_numeric(bike3['운동량'])
bike3['탄소량'] = pd.to_numeric(bike3['탄소량'])

In [ ]:
#운동량 데이터에 이상한 값 존재
#ValueError: Unable to parse string "\N" at position 57
bike3['운동량'][(bike3['운동량'] == r'\N')] = -1
bike3['탄소량'][(bike3['탄소량'] == r'\N')] = -1
#'\'같은 특수문자가 포함되는 경우 unicode error가 발생한다. 이 때는 따옴표 앞에 r을 써준다.

In [ ]:
#대여 일자는 날짜 타입으로 데이터타입을 변경해준다
#날짜 타입은 연, 월, 일, 시, 분, 초, 요일 등의 정보를 보기 편하다
bike3['대여일자'] = pd.to_datetime(bike3['대여일자'])
#df.dtypes

In [ ]:
print(bike3['대여일자'])

##3.데이터 분석하기

###3-1.간단한 분석

In [ ]:
#이동거리(평균)가 큰 순서대로 대여소 명 뽑아보기
bike3.groupby(['대여소명_new'])['이동거리'].mean().reset_index().sort_values(
    by='이동거리', ascending=False).head(3)

In [ ]:
#평균사용시간이 높은 상위 대여소
bike3.groupby(['대여소명_new'])['사용시간'].mean().reset_index().sort_values(
    by='사용시간', ascending=False).head(3)

###3-2. 운동량과 이동거리의 상관관계

In [ ]:
#운동량과 이동거리의 상관관계(runtime ~40s)
#데이터를 좀 더 잘 보이게 그림을 다시 그리자

f, ax = plt.subplots(1, 2, figsize=(18, 8))
sns.scatterplot(data=bike3, x='이동거리', y='운동량', ax=ax[0], hue='성별')
ax[0].set_xlim(-10000, 150000)
ax[0].set_title('이동거리에 따른 운동량(성별)')
sns.scatterplot(data=bike3, x='이동거리', y='운동량',ax=ax[1], hue='연령대코드')
ax[1].set_title('이동거리에 따른 운동량(연령대별)')
plt.show()

###3-3. 대여일자에 따른 남녀 대여건수 비교

In [ ]:
#대여일자 데이터를 년/월/일로 나누기
bike3['대여년'] = bike3['대여일자'].dt.year
bike3['대여월'] = bike3['대여일자'].dt.month
bike3['대여일'] = bike3['대여일자'].dt.day

In [ ]:
bike3.head()

In [ ]:
#성별에 따른 사용시간 합계를 대여일별로 집계하기
bike3.pivot_table(index=bike3['대여일'], columns=bike3['성별'], values='사용시간',  aggfunc=sum)

In [ ]:
#위 정보를 바탕으로 대여 일별 성별에 따른 사용시간 그래프를 그려보자
bike_fm = bike3.pivot_table(index=bike3['대여일'], columns=bike3['성별'], values='사용시간',  aggfunc=sum)
bike_fm.plot(title='일별 자전거 대여수', rot=0)

###3-4. 구별 거치대 개수 지도로 나타내기

In [ ]:
bike1.head()

In [ ]:
#LCD 거치대와 QR 거치대수 합한 새 컬럼 만들기
bike1['총거치대수'] = bike1['거치대수lcd']+bike1['거치대수qr']

In [ ]:
#구별 설치 거치대의 개수
bike_sum1 = bike1.groupby(['자치구'])[('총거치대수')].sum().reset_index()
bike_sum1 = bike_sum1.set_index('자치구')
bike_sum1.index

In [ ]:
#folium으로 서울 지도위에 대여소 분포 나타내기
#folium: 지도 위에 시각화 할 때 유용하다!
import json
import folium
import warnings
#warnings.simplefilter(action = "ignore", category = FutureWarning)

geo_path = '/content/drive/MyDrive/Python/06_Ddareungee/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

[대한민국 folium json map](https://krksap.tistory.com/1751)은 여기서 다운받을 수 있다.

In [ ]:
geo_str

In [ ]:
seoul_bike1 = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
#tiles = 'OpenStreetMap', 'Stamen Terrain'

seoul_bike1.choropleth(
    geo_data = geo_str,
    data = bike_sum1['총거치대수'],
    columns = ['자치구','총거치대수'],
               fill_color = 'YlGnBu',
               key_on = 'feature.properties.name',
               legend_name='Number of Parking lots (%)')

seoul_bike1

###3-5.구별 대여건수 지도로 나타내기

In [ ]:
bike1.info()

In [ ]:
#대여소번호 데이터 타입 변경하기
bike22['대여소번호'] = pd.to_numeric(bike22['대여소번호'])

In [ ]:
#대여소명에 구 붙이기
bike_merge = pd.merge(bike1, bike22, left_on='대여소번호', right_on='대여소번호')

In [ ]:
bike_merge.head()

In [ ]:
#필요한 컬럼만 남기기
bike_merge = bike_merge.drop(['설치시기', '대여소 명'], axis=1, inplace=False)

In [ ]:
#202101 데이터만 사용하기
bike_merge01 = bike_merge[bike_merge['대여 일자 / 월'] == 202101]

In [ ]:
bike_merge01.head(5)

In [ ]:
#자치구별 대여건수, 반납건수 집계하기
bike_sum2 = bike_merge01.groupby(['자치구'])[('대여 건수')].sum().reset_index()
bike_sum2 = bike_sum2.set_index('자치구')

In [ ]:
seoul_bike2 = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='openstreetmap')

seoul_bike2.choropleth(
    geo_data = geo_str,
    data = bike_sum2['대여 건수'],
    columns = [bike_sum2.index,bike_sum2['대여 건수']],
               fill_color = 'PuRd',
               key_on = 'feature.properties.name')

seoul_bike2

##5.Review

* 따릉이 데이터를 살펴본 결과:  
석수역1번출구 앞 (SK주유소)	에서 대여한 따릉이의 이동거리가 가장 크고, IBK기업은행 석수역 지점 앞	대여소 따릉이의 이용시간이 가장 길었다.  
* 따릉이를 이용한 남녀 간 이동거리 분포에 큰 차이는 없었으나, 남성의 운동량이 좀 더 많게 나타났다. WHY?
* 따릉이를 많이 이용하는 성별은?:  전반적으로 남성이 여성보다 따릉이를 더 많이 이용했다(대여건수 기준).
* 서울시 구별 설치 거치대의 개수는?
* 서울시 구별로 거치대 개수와 대여건수는?
